# Feature Engineering and Exploration

This will create monthly features for users, and then explore the best combination of temporal attributes to retain for final data matrix

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from functools import reduce

In [2]:
trx = pd.read_csv('../data/historical_transactions.csv')
trx['new'] = False
merchants = pd.read_csv('../data/merchants.csv')
new_merchant_trx = pd.read_csv('../data/new_merchant_transactions.csv')
new_merchant_trx['new'] = True
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
print(trx.shape)
print(new_merchant_trx.shape)

(29112361, 15)
(1963031, 15)


Need to de-normalize purchase amounts and loyalty scores based on the work of [raddar](https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights)

In [4]:
new_merchant_trx['purchase_amount_new'] = np.round(new_merchant_trx['purchase_amount'] / 0.00150265118 + 497.06,2)
trx['purchase_amount_new'] = np.round(trx['purchase_amount'] / 0.00150265118 + 497.06,2)
train['target_raw'] = 2**train['target']

In [5]:
random.seed(903)
sample = np.random.choice(trx.card_id.unique(), size=1000, replace=False)
trx[trx.card_id.isin(sample)].card_id.unique().size

1000

In [6]:
trx_sample = pd.concat([trx[trx.card_id.isin(sample)], new_merchant_trx[new_merchant_trx.card_id.isin(sample)]])
trx_sample.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,new,purchase_amount_new
21459,N,C_ID_418e92d805,69,N,0,A,367,M_ID_820c7b73c8,-11,-0.724578,2017-03-21 20:37:54,1.0,9,16,False,14.86
21460,Y,C_ID_418e92d805,69,N,0,A,278,M_ID_f86439cec0,-2,-0.727373,2017-12-05 14:58:03,1.0,9,37,False,13.00
21461,N,C_ID_418e92d805,69,N,0,A,705,M_ID_1f4773aa76,-9,-0.735052,2017-05-23 11:23:15,1.0,9,33,False,7.89
21462,Y,C_ID_418e92d805,69,N,0,A,80,M_ID_5ba019a379,-12,-0.596643,2017-02-02 18:13:57,1.0,9,37,False,100.00
21463,Y,C_ID_418e92d805,69,N,0,A,367,M_ID_0f5fd331f8,-1,-0.703526,2018-01-11 10:49:36,1.0,9,16,False,28.87


In [7]:
trx_full = pd.concat([trx, new_merchant_trx])
trx_full.shape

(31075392, 16)

In [8]:
# Cummulative number of unique merchants each month
trx_sample.sort_values(['card_id', 'month_lag']).drop_duplicates(['card_id', 'merchant_id'])\
    .groupby(['card_id', 'month_lag']).merchant_id.size().unstack().fillna(0).cumsum(axis=1)\
    .reset_index().melt(id_vars=['card_id'], var_name='month_lag', value_name='n_total_merchants')\
    .sort_values(['card_id', 'month_lag'])

,card_id,month_lag,n_total_merchants
0,C_ID_003d2dcda8,-13,0.0
1000,C_ID_003d2dcda8,-12,0.0
2000,C_ID_003d2dcda8,-11,0.0
3000,C_ID_003d2dcda8,-10,0.0
4000,C_ID_003d2dcda8,-9,0.0
5000,C_ID_003d2dcda8,-8,0.0
6000,C_ID_003d2dcda8,-7,0.0
7000,C_ID_003d2dcda8,-6,0.0
8000,C_ID_003d2dcda8,-5,0.0
9000,C_ID_003d2dcda8,-4,0.0


In [9]:
# Actual unique merchants newly added each month
monthly_new_merchants = trx_full.sort_values(['card_id', 'month_lag'])\
    .drop_duplicates(['card_id', 'merchant_id'])[['card_id', 'month_lag', 'merchant_id']]
monthly_new_merchants.head(15)

,card_id,month_lag,merchant_id
19095775,C_ID_00007093c1,-12,M_ID_a33355a1b7
19095845,C_ID_00007093c1,-12,M_ID_9400cf2342
19095896,C_ID_00007093c1,-12,M_ID_69423b34e4
19095787,C_ID_00007093c1,-10,M_ID_bc3f2e986e
19095805,C_ID_00007093c1,-10,M_ID_57df19bf28
19095815,C_ID_00007093c1,-10,M_ID_8261f7a734
19095776,C_ID_00007093c1,-9,M_ID_15446e939c
19095842,C_ID_00007093c1,-9,M_ID_3c6556f2dd
19095908,C_ID_00007093c1,-9,M_ID_109e9d2b1d
19095783,C_ID_00007093c1,-8,M_ID_ee8fcd02ca


In [10]:
# Map card_id and month_lag to list of new merchants for that card, month
new_merch_dict = monthly_new_merchants.groupby(['card_id', 'month_lag'])\
    .apply(lambda x: list(x['merchant_id'])).unstack(0).fillna('').to_dict()

In [ ]:
new_merch_dict['C_ID_00007093c1']

In [12]:
# Will subset the group to transactions only at new merchants and sum the amount
def f(group):
    new_merch = list(new_merch_dict.get(group['card_id'].iloc[0]).get(group['month_lag'].iloc[0]))
    return group[group['merchant_id'].isin(new_merch)].purchase_amount_new.sum()

In [13]:
# Group transactions and add total spend and new merchant spend per month per card
grouped = trx_full.groupby(['card_id', 'month_lag'])
groups = pd.DataFrame({'amt_total': grouped.purchase_amount_new.sum()})
groups['amt_new'] = grouped.apply(f)
groups.tail(13)

KeyboardInterrupt: 

We can check the transactions by month, merchant for this specific card (`C_ID_ffe0232b20`) to verify that our aggregation is working correctly. It does appear to be working as intended.

In [ ]:
trx[trx.card_id == 'C_ID_ffe0232b20'].sort_values(['month_lag', 'merchant_id'])\
    [['card_id', 'merchant_id', 'month_lag', 'purchase_amount_new']]

In [ ]:
# Add ratio representing the percent of total spent at new merchants for each month
groups['NDR'] = groups['amt_new'] / groups['amt_total']
groups.head(9)

In [ ]:
# Add counts for number of new merchants added each month
groups['n_new_merchants'] = trx_full.sort_values(['card_id', 'month_lag'])\
    .drop_duplicates(['card_id', 'merchant_id'])\
    .groupby(['card_id', 'month_lag']).merchant_id.size().fillna(0)
groups.head(9)

In [ ]:
# Add counts for total number of cummulative unique merchants each month
groups['n_total_merchants'] = trx_full.sort_values(['card_id', 'month_lag'])\
    .drop_duplicates(['card_id', 'merchant_id'])\
    .groupby(['card_id', 'month_lag']).merchant_id.size()\
    .unstack().fillna(0).cumsum(axis=1)\
    .reset_index().melt(id_vars=['card_id'], var_name='month_lag', value_name='n_total_merchants')\
    .groupby(['card_id', 'month_lag']).first()
groups.head(9)